In [50]:
import json
import sys

import os
print(os.getcwd())
os.chdir("../counta_bot_e2e")

print(os.getcwd())
args = [json.loads(ln) for ln in open("./data/train_cmv_cleaned.jsonl")]


/Users/joshua.sheppard/PycharmProjects/counta_bot_e2e
/Users/joshua.sheppard/PycharmProjects/counta_bot_e2e


In [51]:
import pandas as pd
args_df = pd.DataFrame(args)
len(args_df)

unique = args_df.drop_duplicates(subset=["id"])

unique_ = []
for _, i in unique.iterrows():
    unique_.append({
        "id": i["id"],
        "titles": i["titles"],
        "argument": i["arguments"]
    })

len(unique_)

2509

In [57]:
#from counta_bot.utils.keyphrase_extraction import extract_keyphrase
from yake import KeywordExtractor
import multiprocessing
from tqdm import tqdm

kw_extractor = KeywordExtractor(lan="en", n=3, top=5)

# Note: Alternating on Keyphrase level. Titles vs Arguments
keyphrases = []
with tqdm(total=(len(unique_)), position=0, leave=True) as pbar:
    for arg in unique_:
        # TODOs: Run KeyBERT at Scale
        keyphrases.append([i[0] for i in kw_extractor.extract_keywords(arg["titles"])])
        #keyphrases.append(extract_keyphrase(arg["argument"]))

        pbar.update()

100%|██████████| 2509/2509 [00:04<00:00, 524.98it/s]


In [53]:
print(len(keyphrases))
keyphrases

2509


[['remember the topic',
  'topic that spurred',
  'spurred this discussion',
  'nature',
  'natural'],
 ['recently gotten deeply',
  'deeply involvedobsessed',
  'properties of essential',
  'healing properties',
  'essential oils'],
 ['todays news BBC',
  'BBC WikipediaI',
  'WikipediaI think France',
  'Thinking about todays',
  'tough spot'],
 ['causation', 'view', 'Explaining causation', 'mugged', 'victim blaming'],
 ['philosophy',
  'independent form',
  'study philosophy',
  'study philosophy doesnt',
  'study'],
 ['huge events happen',
  'weeks weve',
  'huge events',
  'events happen',
  'caused by matters'],
 ['human race',
  'talking about making',
  'partially genetic',
  'race smarter forever.Intelligence',
  'human race smarter'],
 ['opposite side',
  'hearing the opposite',
  'post',
  'people',
  'interested in hearing'],
 ['people enjoy driving',
  'difficult to understand',
  'enjoy driving',
  'people enjoy',
  'small'],
 ['Note Please assume', 'die', 'life', 'meaning

In [54]:
from counta_bot.detection.stance_classifier import sentence_stance

stance = []
with tqdm(total=(len(unique_))) as pbar:
    for i, j in zip(unique_, keyphrases):
        if not j:
            stance.append(" ")

        else:
            aspect = j.pop(0)
            stance.append((sentence_stance(i["titles"], aspect), aspect))

        pbar.update()

100%|██████████| 2509/2509 [00:35<00:00, 70.06it/s]


In [55]:
len(stance)

import json
fout = open("../../data/processed_train_cmv.jsonl", "w")

with fout:
    for i, j, k in zip(unique_[0:100], keyphrases, stance):
        fout.write(json.dumps([{
            "id": i["id"],
            "title": i["titles"],
            "argument": i["argument"],
            "keyphrase": j,
            "stance": k
        }]))
        fout.write("\n")

In [56]:
processed = [json.loads(ln) for ln in open("../../data/processed_train_cmv.jsonl")]
processed[0:10]

[[{'id': 't3_2ro9ux',
   'title': 'Anything that is manmade is natural.',
   'argument': 'I cant remember the topic that spurred this discussion but a friend and I were debating whether manmade things were natural. He took the position that they are unnatural. He cited this definition by MerriamWebster existing in nature and not made or caused by people coming from nature as his basis for the distinction for natural vs. unnatural.However I respectfully disagree with his position and furthermore that definition of natural. People arise from nature. Humankinds capacity to create problemsolve analyze rationalize and build also come from natural processes. How are the things we create unnatural? It is only through natural occurrences that we have this ability why is it that we would give the credit of these things solely to man as opposed to nature? We are not separate from nature thus how can any of our actions or creations be unnatural? If we were somehow separate from nature I would und